<img src="https://drive.google.com/uc?export=view&id=10rLzZzH5ctQ4bPRDZTbZk5tQMc9sDO0W" width="100%"></img>

# **Cassandra y Python**
---

En este notebook mostraremos cómo realizar una conexión a una base de datos en _Cassandra_ desde _Python_

## **1. ¿Qué es Cassandra?**
---

<img src="https://drive.google.com/uc?export=view&id=18UJ2ETGf0hZRvJnjpICVOR0fc6HUysR-" width="50%"></img>

* Es una base de datos columnar NoSQL distribuida y basada en modelos de almacenamiento tabulares.
* Fue escrita en _Java_ lo cual permite el manejo de grandes cantidades de datos de forma segura.
* Fue desarrollada en _Facebook_ y lanzada en el 2008, aunque en el 2009 se volvió proyecto de código abierto de tipo Apache.

## **2. Creación de la Base de Datos**
---

Para esta práctica debe crear una cuenta gratuita en [Astra](https://astra.datastax.com/register), puede registrarse con una cuenta de _gmail_, o una cuenta de _GitHub_ o usando su correo personal.

Una vez creada la cuenta debe ingresar la siguiente información sobre el uso de astra:

<img src="https://drive.google.com/uc?export=view&id=1ALOEVX75BSa6QHWesv2uL2WSMOn5RGvS" width="50%">

Con esto, llegará al panel de control general de Astra donde deberá crear una base de datos:

<img src="https://drive.google.com/uc?export=view&id=1hu-GqGgR1ql4OME7zi7yzcrri83WVdil" width="80%">

Debe poner las siguientes configuraciones en la creación de la base de datos:

* **Database name**: `mlds3`
* **Keyspace name**: `test`
* **Provider and region**: puede usar cualquiera de las disponibles, se recomienda usar los valores de la siguiente imagen.

<img src="https://drive.google.com/uc?export=view&id=1ZwlHhc2LhxaeM_jXuTUfdrnoJZ-zdfn1" width="80%">

Luego de crear la base de datos, Astra le proveerá un token con la configuración por defecto, no es necesario que lo descargue, pues más adelante configuraremos más permisos para el acceso a la base de datos. Es decir, puede dar click en **Go to database**.

Ahora, vamos a crear el token de acceso para poder acceder a la base de datos, para ello debe irse a la configuración de organizaciones:

<img src="https://drive.google.com/uc?export=view&id=1rE6-1WHVguK2VWbKrVPgxFaADeTts4xX" width="50%">

Debe irse a la pestaña de administración de tokens donde puede seleccionar distintos tipos de permisos sobre la base de datos. Se recomienda usar el rol autogenerado por astra para los ejercicios de esta unidad como se indica en la siguiente figura. Entre los permisos de este rol encuentra los permisos de creación, lectura, escritura y borrado de tablas que necesitaremos para los ejercicios sobre _Cassandra_.

<img src="https://drive.google.com/uc?export=view&id=1Y3N-yTqJthw6YpIY54-d2iVwF6ahFkIl" width="60%">

Con esto, deberá tener el token que usaremos más adelante para conectarnos a la base de datos, deberá descargar el archivo csv para luego crear la conexión, cambie el nombre de este archivo a `token.csv`.

<img src="https://drive.google.com/uc?export=view&id=19HqbTAqDZdbIvc_9NbWWNVzcUy65le8r" width="60%">

Finalmente, necesitamos un archivo con información del servidor de la base de datos. Para ello, debe volver al inicio de [astra](https://astra.datastax.com/) y hacer click sobre la base de datos que ya tenemos creada:

<img src="https://drive.google.com/uc?export=view&id=1SbZJaO8lhbgPy-f7RtvUPrzmVOMRc1Lm" width="30%">

Ahora, debe hacer click sobre el botón **Connect**:

<img src="https://drive.google.com/uc?export=view&id=1qaV94B4BP-ffCum5w-gQMNfMPGNJAiII" width="100%">

Debe seleccionar el tipo de conexión por **driver** de tipo **legacy** con el lenguaje de programación _Python_. Esto le permitirá descargar un archivo en formato zip para la conexión con la base de datos:

<img src="https://drive.google.com/uc?export=view&id=1BEJ9mbaw7-jyMO0cFbKeNstktQHQHAhO" width="100%">

Debe cambiar el nombre del archivo a `bundle.zip`.

Por último debe subir los archivos `token.csv` y `bundle.zip` a google colaboratory o dejarlos en el espacio de trabajo donde está corriendo el notebook.

## **3. Conexión a la Base de Datos**
---

La conexión a la base de datos se da por medio de la librería `cassandra-driver`, la puede instalar con el siguiente comando:

In [1]:
!pip install cassandra-driver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.8 MB 5.2 MB/s 


In [2]:
!python -c 'import cassandra; print (cassandra.__version__)'

3.25.0


Finalmente, importamos las librerías necesarias para la conexión:

In [3]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import pandas as pd

Para conectarse debe descargar el *bundle* y el token de acceso en formato `csv` que obtuvo al momento de crear la base de datos. Estos archivos los debe subir a google colaboratory.

Veamos cómo crear la conexión:

> **Nota**: debe cambiar el nombre del bundle (formato zip) y del token (formato csv) al nombre que tengan estos archivos en su base de datos.

In [4]:
cloud_config= {
        "secure_connect_bundle": "bundle.zip" # nombre de su archivo de bundle
}

token = pd.read_csv("token.csv")

Ahora, vamos a crear los objetos de conexión y autenticación, creamos los siguientes tres objetos:

* `auth_provider`: objeto de autenticación creado con las credenciales de acceso a [astra](https://astra.datastax.com/).
* `cluster`: objeto de identificación del servidor (cluster) maestro de la base de datos.
* `session`: sesión de conexión sobre la que se ejecutarán los distintos comandos (es equivalente al objeto cursor en bases SQL).

> **Nota**: las bases de datos gratuitas en [astra](https://astra.datastax.com/) se pausan luego de 48h sin uso, debe revisar desde el panel de control de astra que se encuentre activa o en caso contrario resumirla.

In [5]:
auth_provider = PlainTextAuthProvider(
        token["Client Id"].iloc[0],
        token["Client Secret"].iloc[0]
        )
cluster = Cluster(
        cloud=cloud_config,
        auth_provider=auth_provider
        )
session = cluster.connect()

ERROR:cassandra.connection:Closing connection <LibevConnection(139933188226512) 789432d4-2a70-4a5b-842a-0a7bed508af1-us-east1.db.astra.datastax.com:29042:4e3f943d-8c53-46c6-801e-6dabca21a2e0> due to protocol error: Error from server: code=000a [Protocol error] message="Beta version of the protocol used (5/v5-beta), but USE_BETA flag is unset"


## **4. Ejecución de la Consulta**
---

Veamos cómo ejecutar un comando en la base de datos:

In [6]:
query = """
SELECT
    release_version
FROM
    system.local
;
"""
row = session.execute(query).one()
print(row[0])

4.0.0.6816


Este resultado muestra la versión del servidor de Cassandra. Debe obtener un número parecido (no necesariamente igual) al siguiente:

```
4.0.0.6816
```

## **5. Recursos Adicionales**
---

* [AstraDB DataStax](https://www.datastax.com/products/datastax-astra).
* [Python driver for Apache Cassandra](https://docs.datastax.com/en/developer/python-driver/3.25/).

## **6. Créditos**
---

**Profesor**

- [Jorge E. Camargo, PhD](https://dis.unal.edu.co/~jecamargom/)

**Diseño, desarrollo del notebook y material audiovisual**

- [Juan S. Lara MSc](https://www.linkedin.com/in/juan-sebastian-lara-ramirez-43570a214/)

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*